In [3]:
from os import listdir
from random import shuffle, randint
import time

import numpy as np
import tensorflow as tf
import tifffile as tiff
from scipy import ndimage, misc

from loader import Loader
import model as m


input_files = [f for f in listdir('input_data')]
shuffle(input_files)


def data_processor(f, config):
  input_image = tiff.imread('input_data/{0}'.format(f))
  label_image = tiff.imread('label_data/{0}'.format(f[:-1]))
  label_image = label_image[:, :, :1] / 255

  if config.augment:
    angle = randint(0, 360)
    input_image = ndimage.rotate(input_image, angle, reshape=False)
    label_image = ndimage.rotate(label_image, angle, reshape=False)

  input_image = (input_image - np.mean(input_image)) / np.std(input_image)
  return (f, input_image, label_image)


validation_files = input_files[:5]
validation_loader = Loader(validation_files, 5, processor=data_processor)
validation_loader.start()
validation_batch = validation_loader.get_batch(5)
validation_loader.stop()

batch_size = 2
train_files = input_files[5:]
train_loader = Loader(train_files, batch_size * 4, processor=data_processor, randomize=True, augment=True)
train_loader.start()


shouldLoad = False
modelName = m.modelName 

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
config.log_device_placement = True

with tf.Session(config=config) as sess:
  saver = tf.train.Saver(write_version=tf.train.SaverDef.V2)
  summary_writer = tf.summary.FileWriter('summary/{0}'.format(modelName), graph=sess.graph)
  summary_writer_test = tf.summary.FileWriter('summary/{0}-test'.format(modelName), graph=sess.graph)

  if shouldLoad:
    saver.restore(sess, modelName)
  else:
    sess.run(tf.global_variables_initializer())
  
  startTime = time.time()
  print("{0}: training start;".format(startTime))

  while True:
    batch = train_loader.get_batch(batch_size)
    filenames = batch[0]
    input_images = batch[1]
    label_images = batch[2]

    _, error, summary, step = sess.run([m.train, m.error, m.summary, m.global_step], feed_dict={ 
      m.input_image: input_images,  
      m.label_image: label_images
    })

    #if step % 20 == 0:
    #   save_path = saver.save(sess, modelName)
    summary_writer.add_summary(summary, step)
    #print(time.time() - startTime, step, error)
    if step % (100 / batch_size) == 0:
      filenames = validation_batch[0]
      input_images = validation_batch[1]
      label_images = validation_batch[2]

      validation_error, learning_rate, result, summary = sess.run([m.error, m.learning_rate, m.result, m.test_summary], feed_dict={ 
        m.input_image: input_images,  
        m.label_image: label_images,
        m.is_train: False
      })

      summary_writer_test.add_summary(summary, step)

      print("{0}: step {1}; train {2}; test {3}; lrate {4};".format(time.time() - startTime, step, error, validation_error, learning_rate))

      filename = filenames[0]
      result_image = result[0].reshape((1500, 1500))
      misc.imsave("test_results/{0}-{1}-{2}.png".format(modelName, filename, step), result_image)`

    if step%100==0: 
    
        saver.save(sess, 'my_test_model',global_step=step)

    if step == 500: break

train_loader.stop()

1559652835.0993688: training start;
4473.935047149658: step 50; train 0.22849328815937042; test 0.3790343701839447; lrate 0.10000000149011612;


C:\Users\anuhy\Anaconda3\lib\site-packages\ipykernel_launcher.py:97: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


8999.106883764267: step 100; train 0.1842738837003708; test 0.4261179566383362; lrate 0.10000000149011612;
13432.507910490036: step 150; train 0.1560789942741394; test 0.28968310356140137; lrate 0.10000000149011612;
17875.124626874924: step 200; train 0.13615931570529938; test 0.30584654211997986; lrate 0.10000000149011612;
22369.890480279922: step 250; train 0.13196459412574768; test 0.2716556787490845; lrate 0.10000000149011612;
26870.695236444473: step 300; train 0.12725360691547394; test 0.22050590813159943; lrate 0.10000000149011612;
31340.322456121445: step 350; train 0.12848080694675446; test 0.20801280438899994; lrate 0.10000000149011612;
35702.350910425186: step 400; train 0.11230155825614929; test 0.21322666108608246; lrate 0.10000000149011612;
40012.44082379341: step 450; train 0.11761485785245895; test 0.3089674413204193; lrate 0.10000000149011612;
44353.38786697388: step 500; train 0.10624142736196518; test 0.21733519434928894; lrate 0.10000000149011612;


In [6]:
from os import listdir
from random import shuffle, randint
import time

import numpy as np
import tensorflow as tf
import tifffile as tiff
from scipy import ndimage, misc

from loader import Loader
import model as m



def data_processor2(f, config):
  input_image = tiff.imread('test_images/{0}'.format(f))

  if config.augment:
    angle = randint(0, 360)
    label_image = ndimage.rotate(label_image, angle, reshape=False)

  input_image = (input_image - np.mean(input_image)) / np.std(input_image)
  return (f, input_image)

test_files = [filename for filename in listdir('test_images')]
test_loader = Loader(test_files, 1, processor=data_processor2)
test_loader.start()
test_batch = test_loader.get_batch(1)
test_loader.stop()

batch_size=2

shouldLoad = True
modelName = m.modelName

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
config.log_device_placement = True
with tf.Session(config=config) as sess:

    sess.run(tf.global_variables_initializer())
    #graph = tf.get_default_graph()
    new_saver = tf.train.import_meta_graph('my_test_model-500.meta') 
    new_saver.restore(sess, tf.train.latest_checkpoint('./'))
    graph = tf.get_default_graph() 

    startTime = time.time()
    #print("{0}: testing start;".format(startTime))

  
    #print(time.time() - startTime, step, error)
    filenames = test_batch[0]
    input_images = test_batch[1]
    result= sess.run([m.result], feed_dict={ 
        m.input_image: input_images,  
        m.is_train: False
      })

    
    result_image = result[0].reshape((1500, 1500))
    misc.imsave("test_results/result.png", result_image)


  

INFO:tensorflow:Restoring parameters from ./my_test_model-500


C:\Users\anuhy\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
